In [1]:
import os
import sys
import pickle
import numpy as np
import torch 
from tqdm import tqdm
import coremltools

os.environ["CONFIG_PATHS"] = "../configs/small_model.toml,../configs/notebook_overrides.toml"
sys.path.append("../src")

import neural_net
from configuration import config
from neural_net import NeuralNet, ResidualBlock
import player_pov_helpers

""

Torch version 2.5.1 has not been tested with coremltools. You may run into unexpected errors. Torch 2.4.0 is the most recent version that has been tested.


Loaded config:  {"development": {"debug_mode": true, "profile": false, "runtime": 120, "display_logs_in_console": false, "output_directory": "data/"}, "game": {"board_size": 10, "num_moves": 6233, "moves_directory": "../data/moves_10/"}, "network": {"main_body_channels": 64, "value_head_channels": 16, "value_head_flat_layer_width": 64, "policy_head_channels": 64, "residual_blocks": 8}, "architecture": {"gameplay_processes": 6, "coroutines_per_process": 256, "inference_batch_size": 128}, "mcts": {"num_rollouts": 500, "ucb_exploration": 1.4, "ucb_default_child_value": 0.25, "root_dirichlet_alpha": 0.03, "root_exploration_fraction": 0.25}}
Loading file: piece_indices
Loading file: rotation_mapping
Loading file: new_occupieds
Loading file: moves_ruled_out_for_all
Loading file: scores
Loading file: moves_ruled_out_for_player
Loading file: moves_enabled_for_player
Loading file: new_adjacents
Loading file: new_corners


''

In [2]:
BOARD_SIZE = config()["game"]["board_size"]

In [3]:
def model_size(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum(p.numel() for p in model_parameters)
    # return sum([np.prod(p.size()) for p in model_parameters])
    # return sum(p.numel() for p in model.parameters())

In [4]:
import time

def time_per_eval(num_evaluations, batch_size, model):
    num_batches = num_evaluations // batch_size
    num_evaluations = num_batches * batch_size

    random_arrays = np.random.random((num_batches, batch_size, 4, BOARD_SIZE, BOARD_SIZE))
    print("Starting...")

    start = time.perf_counter()
    for i in range(num_batches):
        occupancies = torch.from_numpy(random_arrays[i]).to(device="mps", dtype=torch.float32)
        model(occupancies)
    elapsed = time.perf_counter() - start

    return elapsed / (num_batches * batch_size)

In [5]:
model = NeuralNet()
model_size(model)
print("Model size:", len(model.residual_blocks))

Model size: 8


In [6]:
model.eval()
example_inputs = (torch.rand(16, 4, BOARD_SIZE, BOARD_SIZE),)
exported_program = torch.export.export(model, example_inputs)
model_from_export = coremltools.convert(exported_program, compute_precision=coremltools.precision.FLOAT32)

/Users/shivamsarodia/.pyenv/versions/3.11.3/lib/python3.11/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/Users/shivamsarodia/.pyenv/versions/3.11.3/lib/python3.11/site-packages/onnxscript/converter.py:820: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details p

In [7]:
def time_per_eval_coreml(num_evaluations, batch_size, model):
    num_batches = num_evaluations // batch_size
    num_evaluations = num_batches * batch_size

    random_arrays = np.random.random((num_batches, batch_size, 4, BOARD_SIZE, BOARD_SIZE))

    print("Starting...")
    start = time.perf_counter()
    for i in range(num_batches):
        model.predict({"occupancies": random_arrays[i]})
    elapsed = time.perf_counter() - start

    return elapsed / (num_batches * batch_size)

In [8]:
time_per_eval(
    num_evaluations=10000,
    batch_size=128,
    model=model.to("mps"),
)

Starting...


6.513175240413078e-05

In [11]:
time_per_eval_coreml(
    num_evaluations=10000,
    batch_size=16,
    model=model_from_export,
)

Starting...


0.000142201141701662